# Read in data

In [1]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [2]:
DATA_DIRECTORY = Path("code execution development data/old")

## LAMP

In [3]:
def read_lamp():
    lamp = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
        parse_dates=["timestamp", "forecast_timestamp"],
        dtype={"temperature": "int16", "wind_direction":"int16", "wind_gust":"int16", "cloud_ceiling":"float16", "visibility":"int16"}
    )
    return lamp

## TBFM

In [4]:
def read_tbfm():
    tbfm = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_tbfm.csv.bz2",
        parse_dates=["timestamp", "scheduled_runway_estimated_time"]
    )
    return tbfm

## ETD

In [5]:
def read_etd():
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
        parse_dates=["departure_runway_estimated_time", "timestamp"]
    )
    return etd

## MFS

In [6]:
def read_mfs():
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_mfs.csv.bz2"
    )
    return etd

# Initial Model, Linear Regression

In [7]:
submission_format = pd.read_csv(
    "code execution development data/test_labels.csv", parse_dates=["timestamp"]
)

In [8]:
df = submission_format
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41
2,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,KATL,62
3,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 11:15:00,KATL,17
4,AAL1567.ATL.DFW.201114.1257.0271.TFM,2020-11-15 12:00:00,KATL,53


In [9]:
df.shape

(2046, 4)

## Insert Data

In [10]:
# add empty columns
columns = ['etd',
           'traffic',
           'precip',
           'lightning_prob',
           'cloud',
           'visibility',
           'cloud_ceiling',
           'wind_gust',
           'wind_speed',
           'wind_direction',
           'temperature']

for col in columns:
    df[col] = ''
    
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,lightning_prob,cloud,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,,,,,,,,,,,
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,,,,,,,,,,,
2,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,KATL,62,,,,,,,,,,,
3,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 11:15:00,KATL,17,,,,,,,,,,,
4,AAL1567.ATL.DFW.201114.1257.0271.TFM,2020-11-15 12:00:00,KATL,53,,,,,,,,,,,


In [11]:
def filter_lamp(current_time):
    # weather data
    valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
    return valid_lamp.iloc[-1]
    
def filter_tbfm(current_time):
    # traffic data
    valid_tbfm = tbfm.loc[(tbfm.timestamp <= current_time) 
                    & (tbfm.timestamp > valid_time) ]
    return valid_tbfm['gufi'].nunique()
    
def filter_etd(current_time):    
    #etd data
    valid_etd = etd.loc[(etd.timestamp <= current_time) 
                    & (etd.timestamp > valid_time) 
                    & (etd.gufi == df.loc[i].gufi)]
    try:
        return (valid_etd.iloc[-1].departure_runway_estimated_time - current_time).total_seconds()
    except:
        return 3600

In [12]:
for a in airports:
    
    times = df.timestamp.unique()
    gufis = df.gufi.unique()
    airport = a

    etd = read_etd()
    mfs = read_mfs()
    tbfm = read_tbfm()
    lamp = read_lamp()

    for t in times:
        current_time = pd.to_datetime(t)
        valid_time = current_time - pd.Timedelta(30, unit='hours')
        indices = df[df.timestamp == t].index

        # insert etd data
        for i in indices:
            df.loc[i, 'etd'] = filter_etd(current_time)

        # insert traffic data
        df.loc[indices, 'traffic'] = filter_tbfm(current_time)

        # insert weather data
        df.loc[indices, 'precip'] = filter_lamp(current_time).precip
        df.loc[indices, 'lightning_prob'] = filter_lamp(current_time).lightning_prob
        df.loc[indices, 'cloud'] = filter_lamp(current_time).cloud
        df.loc[indices, 'visibility'] = filter_lamp(current_time).visibility
        df.loc[indices, 'cloud_ceiling'] = filter_lamp(current_time).cloud_ceiling
        df.loc[indices, 'wind_gust'] = filter_lamp(current_time).wind_gust
        df.loc[indices, 'wind_speed'] = filter_lamp(current_time).wind_speed
        df.loc[indices, 'wind_direction'] = filter_lamp(current_time).wind_direction
        df.loc[indices, 'temperature'] = filter_lamp(current_time).temperature

    df.head(5)

C:\Users\Shantel\AppData\Local\Temp\ipykernel_19164\2577368176.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  etd = pd.read_csv(
C:\Users\Shantel\AppData\Local\Temp\ipykernel_19164\2577368176.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  etd = pd.read_csv(


## Feature Engineering

In [13]:
df.nunique()

gufi                      1228
timestamp                   17
airport                     10
minutes_until_pushback     133
etd                         83
traffic                     17
precip                       2
lightning_prob               1
cloud                        1
visibility                   3
cloud_ceiling                4
wind_gust                    3
wind_speed                   6
wind_direction               7
temperature                  4
dtype: int64

In [14]:
# binary encoding
df.replace(False, 0, inplace=True)
df.replace(True, 0, inplace=True)

In [15]:
# nominal encoding
df = pd.get_dummies(df, columns=['cloud', 'lightning_prob'], drop_first=True)
df.head(5)

,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,3600.0,465,0,7,5.0,20,13,19,48
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,3600.0,452,0,7,7.0,0,13,19,48
2,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,KATL,62,3600.0,481,0,5,4.0,22,14,19,48
3,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 11:15:00,KATL,17,3600.0,465,0,7,5.0,20,13,19,48
4,AAL1567.ATL.DFW.201114.1257.0271.TFM,2020-11-15 12:00:00,KATL,53,3600.0,452,0,7,7.0,0,13,19,48


# Linear Regression

## Model 1

In [16]:
import statsmodels.api as sm

Y = df['minutes_until_pushback']
X = df.drop(columns=['gufi', 'timestamp', 'airport', 'minutes_until_pushback'])
X = sm.add_constant(X)

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.125
Model:                                OLS   Adj. R-squared:                  0.122
Method:                     Least Squares   F-statistic:                     36.41
Date:                    Sat, 01 Apr 2023   Prob (F-statistic):           2.66e-54
Time:                            16:31:32   Log-Likelihood:                -9666.3
No. Observations:                    2046   AIC:                         1.935e+04
Df Residuals:                        2037   BIC:                         1.940e+04
Df Model:                               8                                         
Covariance Type:                nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -89.9918     59.459     -1.514      0.130    -206.598      26.615
etd                0.0154      0.001     13.186      0.000       0.013       0.018
traffic            0.2799      0.057      4.944      0.000       0.169       0.391
precip          9.775e-13   6.39e-13      1.529      0.126   -2.76e-13    2.23e-12
visibility        -6.0634      1.758     -3.449      0.001      -9.511      -2.616
cloud_ceiling      5.6085      3.683      1.523      0.128      -1.614      12.831
wind_gust          0.1210      0.401      0.302      0.763      -0.665       0.907
wind_speed         1.3508      0.636      2.123      0.034       0.103       2.598
wind_direction    12.0133      1.821      6.598      0.000       8.443      15.584
temperature       -5.8171      1.776     -3.275      0.001      -9.301      -2.333
==============================================================================
Omnibus:                      641.721   Durbin-Watson:                   2.114
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4169.744
Skew:                           1.314   Prob(JB):                         0.00
Kurtosis:                       9.481   Cond. No.                     5.51e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is  9e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [17]:
# calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

20.472663123664596

In [18]:
# check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,feature,VIF
0,const,9689.033556
1,etd,1.023361
2,traffic,35.626453
3,precip,NaN
4,visibility,6.470206
5,cloud_ceiling,16.181788
6,wind_gust,9.879754
7,wind_speed,5.577822
8,wind_direction,48.795503
9,temperature,18.125800


## Model 2

In [19]:
# reduce features
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# re-define independent variables
X = df.drop(columns=['gufi', 'timestamp', 'airport', 'minutes_until_pushback'])

# linear regression estimator to be used in feature selection
reg = LinearRegression().fit(X, Y)

In [20]:
# reduce features using SequentialFeatureSelector
sfs = SequentialFeatureSelector(reg, n_features_to_select=4)
sfs.fit(X, Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=4)

In [21]:
# return selected features
sfs.get_feature_names_out()

array(['etd', 'precip', 'wind_direction', 'temperature'], dtype=object)

In [22]:
# re-fit model
X = df[['etd', 'precip', 'wind_direction', 'temperature']]
X = sm.add_constant(X)

model2 = sm.OLS(Y,X).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.112
Model:                                OLS   Adj. R-squared:                  0.111
Method:                     Least Squares   F-statistic:                     85.77
Date:                    Sat, 01 Apr 2023   Prob (F-statistic):           2.86e-52
Time:                            16:31:32   Log-Likelihood:                -9681.6
No. Observations:                    2046   AIC:                         1.937e+04
Df Residuals:                        2042   BIC:                         1.939e+04
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             55.0558     32.720      1.683      0.093      -9.112     119.224
etd                0.0158      0.001     13.465      0.000       0.013       0.018
precip          4.798e-16   1.18e-16      4.059      0.000    2.48e-16    7.12e-16
wind_direction     3.4876      0.539      6.474      0.000       2.431       4.544
temperature       -2.5479      0.865     -2.947      0.003      -4.244      -0.852
==============================================================================
Omnibus:                      639.581   Durbin-Watson:                   2.189
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3952.307
Skew:                           1.325   Prob(JB):                         0.00
Kurtosis:                       9.272   Cond. No.                     9.03e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.26e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [23]:
# re-calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

20.472663123664596

In [24]:
# re-check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,feature,VIF
0,const,2897.566636
1,etd,1.011563
2,precip,NaN
3,wind_direction,4.218056
4,temperature,4.240964
